### 1 Строим запрос к нашему сервису

In [1]:
import requests

import urllib.request
import json 

import pandas as pd
from urllib import request, parse

import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score

In [2]:
# Пример данных
data = (
    19810,
    60.3,
    50
)

In [3]:
# формируем запрос
def send_json(x):
    age, weight, ap_lo  = x
   
    body = {
        'age': age, 
        'weight': weight,
        'ap_lo': ap_lo
        }
   
 #   print(body)

    myurl = 'http://127.0.0.1:8082/' + '/predict'
    headers = {'content-type': 'application/json'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [4]:
# обращение к серверу с запросом из одного набора data
response = send_json(data)
print('prediction', response)

prediction 0.41091756342829266


### 2 Сделаем обработку массы запросов, загрузив X_test, y_test

In [5]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

In [6]:
X_test[['age', 'weight', 'ap_lo']]

,age,weight,ap_lo
0,15216,61.0,80
1,19815,54.0,80
2,20563,69.0,80
3,16625,57.0,70
4,14451,110.0,80
...,...,...,...
17495,19616,63.0,80
17496,21149,54.0,80
17497,15972,64.0,70
17498,18814,63.0,90


Передаем запрос одиночный из наших данных и посмотрим на ответ

In [7]:
response = send_json(X_test[['age', 'weight', 'ap_lo']].iloc[1,:])

In [8]:
response

0.38517198179955026

### 3 Сделаем N запросов и оценим время

In [9]:
N = 100

In [10]:
%%time
predictions = X_test[['age', 'weight', 'ap_lo']].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: total: 562 ms
Wall time: 2.61 s


In [11]:
predictions.values[:5]

array([0.2296422 , 0.38517198, 0.51611893, 0.25971371, 0.47138702])

### 4 Посчитаем метрику

In [12]:
y_test.head()

,cardio
0,0
1,0
2,0
3,0
4,0


In [13]:
y_test.size

17500

In [14]:
y_test.value_counts()

cardio
0         8820
1         8680
dtype: int64

In [15]:
precision, recall, thresholds = precision_recall_curve(y_test[:N], predictions)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print(f'Best Threshold={thresholds[ix]}, F-Score={fscore[ix]:.3f}, Precision={precision[ix]:.3f}, Recall={recall[ix]:.3f}')

Best Threshold=0.25992135462643295, F-Score=0.657, Precision=0.490, Recall=1.000


In [16]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.6354877559213168